In [ ]:
import numpy as np

from lac.utils.camera import Camera
from lac.utils.plotting import plot_surface, plot_poses, plot_3d_points

%load_ext autoreload
%autoreload 2

In [ ]:
# Generate a set of z=f(x,y) points according to sin(x)*sin(y)
n = 100
x = np.linspace(0, 2 * np.pi, n)
y = np.linspace(0, 2 * np.pi, n)
X, Y = np.meshgrid(x, y)
Z = np.cos(X) * np.cos(Y)
color = Z
grid = np.stack([X, Y, Z], axis=-1)

plot_surface(grid)

In [ ]:
camera_t = np.array([0, 3, 1])
camera_R = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]).T
camera_pose = np.eye(4)
camera_pose[:3, :3] = camera_R
camera_pose[:3, 3] = camera_t

cam = Camera(camera_pose)

camera_pose_inv = np.linalg.inv(camera_pose)

In [ ]:
fig = plot_surface(grid)
fig = plot_poses([camera_pose], fig=fig)
fig.show()

In [ ]:
from lac.params import CAMERA_INTRINSICS

K = CAMERA_INTRINSICS

In [ ]:
# Transform the grid to camera frame
points = np.stack([X.flatten(), Y.flatten(), Z.flatten(), np.ones_like(X.flatten())], axis=-1)
points_camera = points @ camera_pose_inv.T

In [ ]:
points = np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=-1)
points.shape

In [ ]:
np.hstack((points, np.ones((points.shape[0], 1))))

In [ ]:
points_camera = cam.project_world_points_to_camera(points)
points_camera

In [ ]:
fig = plot_3d_points(points_camera, color=color)
fig = plot_poses([np.eye(4)], fig=fig)
fig.show()

In [ ]:
uvw = points_camera[:, :3] @ K.T
uv = uvw[:, :2] / uvw[:, 2:]
depths = uvw[:, 2]
color = points_camera[:, 1]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(uv[:, 0], uv[:, 1], c=color, s=1, cmap="plasma")
# Set x and y limits
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
out_of_frame_idxs = np.logical_or(
    np.logical_or(uv[:, 0] < 0, uv[:, 0] > 1280), np.logical_or(uv[:, 1] < 0, uv[:, 1] > 720)
)
out_of_frame_idxs = np.logical_or(out_of_frame_idxs, depths < 0)
out_of_frame_idxs = np.logical_or(
    out_of_frame_idxs, (~np.isfinite(uv[:, 0])) | (~np.isfinite(uv[:, 1]))
)

uv_inframe = uv[~out_of_frame_idxs]
depths_inframe = depths[~out_of_frame_idxs]
color_inframe = color[~out_of_frame_idxs]

In [ ]:
uv_inframe, depths_inframe, color_inframe = cam.project_world_points_to_uv(points, color)

In [ ]:
# Sort the points by depth
sorted_indices = np.argsort(depths_inframe)
uv_sorted = uv_inframe[sorted_indices]
depths_sorted = depths_inframe[sorted_indices]
color_sorted = color_inframe[sorted_indices]

In [ ]:
c = plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c=depths_sorted[::-1], cmap="plasma")
plt.colorbar(c)
# Set x and y limits
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
blank_image = np.zeros((720, 1280)) * np.nan
pixel_set = np.zeros_like(blank_image, dtype=bool)

pt_size_u = 20
pt_size_v = 2

for idx in sorted_indices:
    u, v = uv_inframe[idx]

    if pixel_set[int(v), int(u)]:
        continue
    # Color a 5x5 patch around the point
    for u_i in range(int(u - pt_size_u / 2), int(u + pt_size_u / 2)):
        for v_j in range(int(v - pt_size_v / 2), 720):
            if 0 <= u_i < 1280 and 0 <= v_j < 720:
                if not pixel_set[v_j, u_i]:
                    blank_image[int(v_j), int(u_i)] = color_inframe[idx]
                    pixel_set[v_j, u_i] = True


fig = plt.figure(figsize=(20, 10))
c = plt.imshow(blank_image, cmap="plasma")
plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c="black", alpha=0.1, cmap="plasma")
plt.colorbar(c)
plt.show()